In [1]:
import QuantLib as ql
import pandas as pd

In [2]:
today = ql.Date(8, ql.October, 2014)
ql.Settings.instance().evaluationDate = today

In [3]:
forecast_curve = ql.RelinkableYieldTermStructureHandle()
forecast_curve.linkTo(ql.FlatForward(today, 0.002, ql.Actual360(), ql.Compounded, ql.Semiannual))

In [4]:
index = ql.Euribor6M(forecast_curve)
index.addFixing(ql.Date(6, ql.August, 2014), 0.002)

In [5]:
issueDate = ql.Date(8, ql.August, 2014)
maturityDate = ql.Date(8, ql.August, 2019)

schedule = ql.Schedule(issueDate, maturityDate, ql.Period(ql.Semiannual), ql.TARGET(), ql.Following, ql.Following, ql.DateGeneration.Backward, False)

bond = ql.FloatingRateBond(settlementDays=3, faceAmount=100, schedule=schedule, index=index, paymentDayCounter=ql.Actual360())

In [7]:
dates = [c.date() for c in bond.cashflows()]
cfs = [c.amount() for c in bond.cashflows()]
pd.DataFrame(list(zip(dates, cfs)), columns=('date', 'amount'), index=range(1, len(dates)+1))

,date,amount
1,"February 9th, 2015",0.102778
2,"August 10th, 2015",0.101112
3,"February 8th, 2016",0.101112
4,"August 8th, 2016",0.101112
5,"February 8th, 2017",0.102223
6,"August 8th, 2017",0.100556
7,"February 8th, 2018",0.102223
8,"August 8th, 2018",0.100556
9,"February 8th, 2019",0.102223
10,"August 8th, 2019",0.100556


In [8]:
y = ql.InterestRate(0.002, ql.Actual360(), ql.Compounded, ql.Semiannual)
print(ql.BondFunctions.duration(bond, y, ql.Duration.Modified))

4.8609591731332165


In [9]:
y = ql.SimpleQuote(0.002)
yield_curve = ql.FlatForward(bond.settlementDate(), ql.QuoteHandle(y), ql.Actual360(), ql.Compounded, ql.Semiannual)

dates = [c.date() for c in bond.cashflows()]
cfs = [c.amount() for c in bond.cashflows()]
discounts = [yield_curve.discount(d) for d in dates]
P = sum(cf*b for cf, b in zip(cfs, discounts))

In [10]:
print(P)

bond.setPricingEngine(ql.DiscountingBondEngine(ql.YieldTermStructureHandle(yield_curve)))
print(bond.dirtyPrice())

100.03665363580889
100.03665363580889


In [12]:
dy = 1e-5

y.setValue(0.002+dy)
cfs_p = [c.amount() for c in bond.cashflows()]
discounts_p = [yield_curve.discount(d) for d in dates]
P_p = sum(cf*b for cf, b in zip(cfs_p, discounts_p))
print(P_p)

y.setValue(0.002-dy)
cfs_m = [c.amount() for c in bond.cashflows()]
discounts_m = [yield_curve.discount(d) for d in dates]
P_m = sum(cf*b for cf, b in zip(cfs_m, discounts_m))
print(P_m)

y.setValue(0.002)

100.03179102561501
100.0415165074028


In [13]:
print(-(1/P)*(P_p - P_m)/(2*dy))

4.8609591756253225


In [14]:
pd.DataFrame(list(zip(dates, cfs, discounts, cfs_p, discounts_p, cfs_m, discounts_m)), 
             columns=('date', 'amount', 'discounts', 'amount (+)', 'discounts (+)', 'amount (-)', 'dsicounts (-)'),
             index = range(1, len(dates)+1))

,date,amount,discounts,amount (+),discounts (+),amount (-),dsicounts (-)
1,"February 9th, 2015",0.102778,0.999339,0.102778,0.999336,0.102778,0.999343
2,"August 10th, 2015",0.101112,0.998330,0.101112,0.998322,0.101112,0.998338
3,"February 8th, 2016",0.101112,0.997322,0.101112,0.997308,0.101112,0.997335
4,"August 8th, 2016",0.101112,0.996314,0.101112,0.996296,0.101112,0.996333
5,"February 8th, 2017",0.102223,0.995297,0.102223,0.995273,0.102223,0.995320
6,"August 8th, 2017",0.100556,0.994297,0.100556,0.994269,0.100556,0.994325
7,"February 8th, 2018",0.102223,0.993282,0.102223,0.993248,0.102223,0.993315
8,"August 8th, 2018",0.100556,0.992284,0.100556,0.992245,0.100556,0.992322
9,"February 8th, 2019",0.102223,0.991270,0.102223,0.991227,0.102223,0.991314
10,"August 8th, 2019",0.100556,0.990275,0.100556,0.990226,0.100556,0.990323


In [15]:
forecast_curve.linkTo(yield_curve)

In [16]:
y.setValue(0.002+dy)
cfs_p = [c.amount() for c in bond.cashflows()]
discounts_p = [yield_curve.discount(d) for d in dates]
P_p = sum(cf*b for cf, b in zip(cfs_p, discounts_p))
print(P_p)

y.setValue(0.002-dy)
cfs_m = [c.amount() for c in bond.cashflows()]
discounts_m = [yield_curve.discount(d) for d in dates]
P_m = sum(cf*b for cf, b in zip(cfs_m, discounts_m))
print(P_m)

y.setValue(0.002)

100.03632329080955
100.03698398354918


In [17]:
pd.DataFrame(list(zip(dates, cfs, discounts, cfs_p, discounts_p, cfs_m, discounts_m)), 
             columns=('date', 'amount', 'discounts', 'amount (+)', 'discounts (+)', 'amount (-)', 'dsicounts (-)'),
             index = range(1, len(dates)+1))

,date,amount,discounts,amount (+),discounts (+),amount (-),dsicounts (-)
1,"February 9th, 2015",0.102778,0.999339,0.102778,0.999336,0.102778,0.999343
2,"August 10th, 2015",0.101112,0.998330,0.101617,0.998322,0.100606,0.998338
3,"February 8th, 2016",0.101112,0.997322,0.101617,0.997308,0.100606,0.997335
4,"August 8th, 2016",0.101112,0.996314,0.101617,0.996296,0.100606,0.996333
5,"February 8th, 2017",0.102223,0.995297,0.102734,0.995273,0.101712,0.995320
6,"August 8th, 2017",0.100556,0.994297,0.101059,0.994269,0.100053,0.994325
7,"February 8th, 2018",0.102223,0.993282,0.102734,0.993248,0.101712,0.993315
8,"August 8th, 2018",0.100556,0.992284,0.101059,0.992245,0.100053,0.992322
9,"February 8th, 2019",0.102223,0.991270,0.102734,0.991227,0.101712,0.991314
10,"August 8th, 2019",0.100556,0.990275,0.101059,0.990226,0.100053,0.990323


In [18]:
print(-(1/P)*(P_p - P_m)/(2*dy))

0.33022533022465994


In [19]:
discount_curve = ql.ZeroSpreadedTermStructure(forecast_curve, ql.QuoteHandle(ql.SimpleQuote(0.001)))
bond.setPricingEngine(ql.DiscountingBondEngine(ql.YieldTermStructureHandle(discount_curve)))

In [21]:
P = bond.dirtyPrice()
cfs = [c.amount() for c in bond.cashflows()]
discounts = [discount_curve.discount(d) for d in dates]
print(P)

99.55107926688962


In [22]:
pd.DataFrame(list(zip(dates, cfs, discounts)), columns=('date', 'amount', 'discount'), index=range(1, len(dates)+1))

,date,amount,discount
1,"February 9th, 2015",0.102778,0.999009
2,"August 10th, 2015",0.101112,0.997496
3,"February 8th, 2016",0.101112,0.995984
4,"August 8th, 2016",0.101112,0.994475
5,"February 8th, 2017",0.102223,0.992952
6,"August 8th, 2017",0.100556,0.991456
7,"February 8th, 2018",0.102223,0.989938
8,"August 8th, 2018",0.100556,0.988446
9,"February 8th, 2019",0.102223,0.986932
10,"August 8th, 2019",0.100556,0.985445


In [23]:
y.setValue(0.002+dy)
P_p = bond.dirtyPrice()
print(P_p)

y.setValue(0.002-dy)
P_m = bond.dirtyPrice()
print(P_m)

y.setValue(0.002)

99.55075966035385
99.55139887578544


In [24]:
print(-(1/P)*(P_p - P_m)/(2*dy))

0.3210489711903113
